###### By Phaiboon Jaradnaparatana 6434467223

# import library and stock information

In [119]:
import itertools
import sys
import numpy as np
import pandas as pd
import os
import re
from docx import Document
from docx.shared import Pt

from SolPort import StockData, ComponentsCal, SolutionComputation, stock_info # ?stock_info move to __init__ file in SolPort

np.set_printoptions(threshold=sys.maxsize, linewidth=1000, precision=6, suppress=True)

# Clean data, Calculate dividend_yield

In [2]:
years = [2017, 2018, 2019, 2020, 2021]
stock_data = StockData(stock_info, years)
dividend_yield_matrix, dividend_yield_df = stock_data.calculate_dividend_yields()
print(dividend_yield_df.to_string())
print(f"""\nSelected stocks: ["ENB", "AVGO", "SIRI", "ORI", "ICHI"]""")
dividend_yield_df = dividend_yield_df.loc[:, ["ENB", "AVGO", "SIRI", "ORI", "ICHI"]]
print(dividend_yield_df.to_string())

         SAPPE       UNH        VZ        ENB       PTT        UL       KTB      MSFT       SAT      TSMC       SCB       JPM      Visa  META       TOP      BMWG        AP      ABBV       KBS       NVDA  Cpaxt     APPLE       AVGO        LH       SIRI       CVX        ORI       PSEC       KCE       HCA     TISCO  AMZN      ICHI         O
2017  3.316583  1.500102  5.546715  11.800000  4.540598  3.061825  3.829787  0.658710  8.638957  2.924083  4.892368  3.490401  0.621272   0.0  3.082437  4.785343  5.369128  4.833973  3.738318  17.927171    0.0  1.394495  35.443038  6.632653   7.339450  4.883554  10.161871  11.180124  1.665973  0.299536  7.045862   0.0  0.990099  3.807989
2018  4.300518  1.377324  5.432640  13.556485  4.709660  3.186319  6.756757  0.272123  8.196721  1.279742  4.910479  3.824092  1.015658   0.0  0.961538  3.461166  6.206897  4.405040  2.238806  10.865874    0.0  0.687715  30.486412  7.547170   4.938272  7.148795   4.341317  13.308688  2.820874  0.346942  8.754384   0.0 

## Calculate covariance matrix & bordered hessian matrix
- Combination 5 stocks from 34 stocks
- Calculate covariance matrix
- Calculate bordered hessian matrix

In [3]:
# Choose 5 stocks from 34 to analyze by permutation
stocks = list(stock_info.keys())
combinations = list(itertools.combinations(stocks, 5))

print(f'Total number of combinations : {len(combinations)}')
print(f'First 5 combinations :\n{np.array(combinations[:5])}') 

Total number of combinations : 278256
First 5 combinations :
[['SAPPE' 'UNH' 'VZ' 'ENB' 'PTT']
 ['SAPPE' 'UNH' 'VZ' 'ENB' 'UL']
 ['SAPPE' 'UNH' 'VZ' 'ENB' 'KTB']
 ['SAPPE' 'UNH' 'VZ' 'ENB' 'MSFT']
 ['SAPPE' 'UNH' 'VZ' 'ENB' 'SAT']]


In [4]:
H_matrices = {}

results = ComponentsCal.run_parallel_calculations(combinations, dividend_yield_matrix, stocks)

for combination, covariance_matrix, bordered_hessian in results:
    H_matrices[combination] = (covariance_matrix, bordered_hessian)

100%|██████████| 278256/278256 [00:18<00:00, 15315.86it/s]


In [5]:
import pprint
print(f"""H_matrices of ("ENB", "AVGO", "LH", "ORI", "ICHI"): """)
print(f"""
first index is covariance matrix
    second index is mean values
        third index is bordered hessian
      """)
pprint.pprint(H_matrices[("ENB", "AVGO", "SIRI", "ORI", "ICHI")])

H_matrices of ("ENB", "AVGO", "LH", "ORI", "ICHI"): 

first index is covariance matrix
    second index is mean values
        third index is bordered hessian
      
(array([[  2.179898,   3.592748,  -2.219955,  -0.253275,  -1.232053],
       [  3.592748,  53.426438,  -8.146958,  -0.020412, -14.854652],
       [ -2.219955,  -8.146958,   7.222506,   3.361118,   2.599484],
       [ -0.253275,  -0.020412,   3.361118,   6.565637,  -1.484361],
       [ -1.232053, -14.854652,   2.599484,  -1.484361,   4.758632]]),
 array([[  2.179898,   3.592748,  -2.219955,  -0.253275,  -1.232053,  -1.      ,   0.      ,   0.      ,   0.      ,   0.      ,   1.      ,  11.315467],
       [  3.592748,  53.426438,  -8.146958,  -0.020412, -14.854652,   0.      ,  -1.      ,   0.      ,   0.      ,   0.      ,   1.      ,  27.12289 ],
       [ -2.219955,  -8.146958,   7.222506,   3.361118,   2.599484,   0.      ,   0.      ,  -1.      ,   0.      ,   0.      ,   1.      ,   7.395586],
       [ -0.253275,  -0.02

In [6]:
solutions_file = '../out/raw_data/valid_solutions10Percentv4.pkl'
# Initialize the SolutionComputation class
want_to_print = ("ENB", "AVGO", "SIRI", "ORI", "ICHI")
solution_computer = SolutionComputation(H_matrices, r = 10, want_to_print = want_to_print)

if os.path.exists(solutions_file):
    print(f"Loading solutions from {solutions_file}")
    solution_computer.load_results(solutions_file)
    print("Solutions loaded successfully")
else:
    solution_computer.compute_Valid_Solutions()
    print(f"Saving valid solutions to {solutions_file}...")
    solution_computer.save_results(solutions_file)
    print("Solutions saved successfully")
    
valid_solutions = solution_computer.get_valid_solutions()

Processing tasks: 100%|██████████| 8904192/8904192 [03:15<00:00, 45553.99it/s]


Filtering out None results...
Successfully filtered out None results!
Filtering valid solutions... (lambda>=0, c>=0)


Filtering valid solutions: 100%|██████████| 278256/278256 [01:46<00:00, 2603.51it/s]


Successfully filtered valid solutions!
Saving valid solutions to ../out/raw_data/valid_solutions10Percentv4.pkl...
Solutions saved successfully


In [7]:
print(f"Total number of solutions: {len(valid_solutions)}")
# print the first solution for debugging
for combination, solution_list in valid_solutions.items():
    if combination == ("ENB", "AVGO", "SIRI", "ORI", "ICHI"):
        print(f"Combination : {combination}")
        print(f"Number of solutions : {len(solution_list)}")
        print("Last solution : ", solution_list[-1])
        break

Total number of solutions: 110944
Combination : ('ENB', 'AVGO', 'SIRI', 'ORI', 'ICHI')
Number of solutions : 1
Last solution :  (array([[0.209964],
       [0.174273],
       [0.026621],
       [0.081767],
       [0.507375]]), 0.46714252150862545, array([[  2.179898,   3.592748,  -2.219955,  -0.253275,  -1.232053],
       [  3.592748,  53.426438,  -8.146958,  -0.020412, -14.854652],
       [ -2.219955,  -8.146958,   7.222506,   3.361118,   2.599484],
       [ -0.253275,  -0.020412,   3.361118,   6.565637,  -1.484361],
       [ -1.232053, -14.854652,   2.599484,  -1.484361,   4.758632]]), array([[  2.179898,   3.592748,  -2.219955,  -0.253275,  -1.232053,  -1.      ,   0.      ,   0.      ,   0.      ,   0.      ,   1.      ,  11.315467],
       [  3.592748,  53.426438,  -8.146958,  -0.020412, -14.854652,   0.      ,  -1.      ,   0.      ,   0.      ,   0.      ,   1.      ,  27.12289 ],
       [ -2.219955,  -8.146958,   7.222506,   3.361118,   2.599484,   0.      ,   0.      ,  -1.    

# Print Solution
- Convert want_to_print to Latex

In [132]:
mySolution = solution_computer.get_string_want_to_print()
print(mySolution)

def convert_to_latex(output_string):
    # Step 1: Replace ci_case with LaTeX-friendly format
    latex_string = re.sub(r'ci_case:\((.*?)\)', r'\\text{ci\\_case}:(\1)', output_string)

    # Step 2: Replace matrices with LaTeX bmatrix format inside align* environment
    # Find all matrix blocks (start with [[ and end with ]])
    def matrix_to_latex(match):
        matrix_text = match.group(0)

        # Replace brackets and formatting for LaTeX bmatrix
        matrix_text = matrix_text.replace("[[", "\\begin{bmatrix}")
        matrix_text = matrix_text.replace("]]", "\\end{bmatrix}")
        matrix_text = matrix_text.replace("[", "")  # Remove extra opening brackets
        matrix_text = matrix_text.replace("]", "")  # Remove extra closing brackets
        
        # Split rows based on newline or row separators and clean up spaces
        rows = matrix_text.split("\n")
        formatted_rows = []
        
        for row in rows:
            row = row.strip()  # Strip extra spaces
            if 'e+' not in row:
                # Split row elements by spaces and join them with `&`
                row_elements = row.split()
                # convert 1. or 0. to 1 or 0 only '1.' or '0.' is considered not in the middle of a number
                for i in row_elements:
                    if i == '1.':
                        row_elements[row_elements.index(i)] = '1'
                    elif i == '0.':
                        row_elements[row_elements.index(i)] = '0'
                    elif i == '-1.':
                        row_elements[row_elements.index(i)] = '-1'
                formatted_row = " & ".join(row_elements)
                if formatted_row.startswith("\\begin{bmatrix}"):
                    formatted_row = formatted_row.replace("\\begin{bmatrix} &", "\\begin{bmatrix}")
                if formatted_row.endswith("& \\end{bmatrix}"):
                    formatted_row = formatted_row.replace("& \\end{bmatrix}", "\\end{bmatrix}")
                formatted_rows.append(formatted_row)
            else :
                return f""
            
        matrix_text = " \\\\\n".join(formatted_rows)
        # Wrap the matrix in the align* environment
        return f"\\begin{{align*}}\n{matrix_text}\n\\end{{align*}}"
    # Use regex to find matrices and convert them
    latex_string = re.sub(r'\[\[.*?\]\]', matrix_to_latex, latex_string, flags=re.DOTALL)

    return latex_string

Combinations:
ci_case:(0, 0, 0, 0, 0)
[[-1.        0.        0.        0.        0.        1.       11.315467]
 [ 0.       -1.        0.        0.        0.        1.       27.12289 ]
 [ 0.        0.       -1.        0.        0.        1.        7.395586]
 [ 0.        0.        0.       -1.        0.        1.        6.710784]
 [ 0.        0.        0.        0.       -1.        1.        4.241006]
 [ 0.        0.        0.        0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.        0.        0.        0.      ]]
ci_case:(0, 0, 0, 0, 1)
[[ -1.232053  -1.         0.         0.         0.         1.        11.315467]
 [-14.854652   0.        -1.         0.         0.         1.        27.12289 ]
 [  2.599484   0.         0.        -1.         0.         1.         7.395586]
 [ -1.484361   0.         0.         0.        -1.         1.         6.710784]
 [  4.758632   0.         0.         0.         0.         1.         4.241006]
 [  1.         0.         

In [133]:
latex_output = convert_to_latex(mySolution)
print(latex_output)

Combinations:
\text{ci\_case}:(0, 0, 0, 0, 0)
\begin{align*}
\begin{bmatrix}-1. & 0 & 0 & 0 & 0 & 1 & 11.315467 \\
0 & -1 & 0 & 0 & 0 & 1 & 27.12289 \\
0 & 0 & -1 & 0 & 0 & 1 & 7.395586 \\
0 & 0 & 0 & -1 & 0 & 1 & 6.710784 \\
0 & 0 & 0 & 0 & -1 & 1 & 4.241006 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}
\end{align*}
\text{ci\_case}:(0, 0, 0, 0, 1)
\begin{align*}
\begin{bmatrix} -1.232053 & -1 & 0 & 0 & 0 & 1 & 11.315467 \\
-14.854652 & 0 & -1 & 0 & 0 & 1 & 27.12289 \\
2.599484 & 0 & 0 & -1 & 0 & 1 & 7.395586 \\
-1.484361 & 0 & 0 & 0 & -1 & 1 & 6.710784 \\
4.758632 & 0 & 0 & 0 & 0 & 1 & 4.241006 \\
1 & 0 & 0 & 0 & 0 & 0 & 0 \\
4.241006 & 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}
\end{align*}

\text{ci\_case}:(0, 0, 0, 1, 0)
\begin{align*}
\begin{bmatrix}-0.253275 & -1 & 0 & 0 & 0 & 1 & 11.315467 \\
-0.020412 & 0 & -1 & 0 & 0 & 1 & 27.12289 \\
3.361118 & 0 & 0 & -1 & 0 & 1 & 7.395586 \\
6.565637 & 0 & 0 & 0 & 0 & 1 & 6.710784 \\
-1.484361 & 0 & 0 & 0 & -1 & 1 & 4.241

# Convert to excel file

In [9]:
# Assuming valid_solutions is already defined
data = []  # List to store the rows for the DataFrame

for combination, solution_list in valid_solutions.items():
    for solution in solution_list:
        myPort = solution[0]
        Risk = solution[1]
        Covariance_Matrix = solution[2]
        Hessian = solution[3]
        Hessian_reduced = solution[4]
        ci_case = solution[5]
        x_reduced = solution[6]
        checker = solution[7]
        mu_values = solution[8]
        data.append([combination, myPort, Risk, Covariance_Matrix, Hessian,Hessian_reduced, ci_case, x_reduced, checker, mu_values])
# Create a DataFrame
df_valid_solutions = pd.DataFrame(data, columns=['Combination', 'Portfolio', 'Risk', 'Covariance_Matrix', 'Hessian Matrix','Hessian_reduced', 'ci_case', 'x_reduced', 'checker', 'mu_values'])
# Display the DataFrame
print(df_valid_solutions)


                           Combination  \
0           (SAPPE, UNH, VZ, ENB, TOP)   
1          (SAPPE, UNH, VZ, ENB, BMWG)   
2           (SAPPE, UNH, VZ, ENB, KBS)   
3          (SAPPE, UNH, VZ, ENB, NVDA)   
4          (SAPPE, UNH, VZ, ENB, AVGO)   
...                                ...   
111536   (ENB, SCB, META, Cpaxt, AMZN)   
111537   (ENB, JPM, META, Cpaxt, AMZN)   
111538  (ENB, Visa, META, Cpaxt, AMZN)   
111539  (ENB, Visa, META, Cpaxt, AMZN)   
111540  (ENB, Visa, META, Cpaxt, AMZN)   

                                                Portfolio      Risk  \
0       [[0.0], [0.0], [0.0], [0.8288217586152385], [0...  0.884618   
1       [[0.0], [0.0], [0.13878570588027261], [0.76379...  0.976338   
2       [[0.0], [0.0], [0.25515876213315436], [0.74484...  0.986816   
3       [[0.0], [0.0], [0.2551587621331545], [0.744841...  0.986816   
4       [[0.0], [0.0], [0.5418992793393675], [0.364582...  0.682524   
...                                                   ...       ...  

In [10]:
df_valid_solutions.to_excel('../out/Solutions10percentv4.xlsx', index=False)